adapted from :https://medium.com/@chand.shelvin/pybullet-getting-started-a068a0e3d492

http://www.mymodelrobot.appspot.com/5674976526991360

https://wiki.ros.org/urdf/XML/link

https://www.programcreek.com/python/example/122099/pybullet.setJointMotorControl2

In [3]:
import pybullet as p
import time
import pybullet_data

In [4]:
import random as r
from urdfpy import URDF
import urdfpy
import numpy as np

In [36]:
physicsClient = p.connect(p.GUI)# p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) # optionally
p.setGravity(0,0,-9.8)   
planeId = p.loadURDF('plane.urdf')
cubeStartPos = [0,0,1]
cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF('starfish.urdf',cubeStartPos, cubeStartOrientation)
p.setJointMotorControl2(boxId,0,p.POSITION_CONTROL,-1)
p.setJointMotorControl2(boxId,1,p.POSITION_CONTROL,-1)
p.setJointMotorControl2(boxId,2,p.POSITION_CONTROL,1)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print("start")
print(cubePos,cubeOrn)
p.resetDebugVisualizerCamera( cameraDistance=3.5, cameraYaw=30, cameraPitch=52, cameraTargetPosition=cubePos)
for i in range (300):
   p.stepSimulation()
   time.sleep(1./200.)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)
 

start
(0.0, 0.0, 1.0) (0.0, 0.0, 0.0, 1.0)
(-0.00023944807718888006, -0.014229996925550797, 0.25331723973251863) (0.050204083161644035, -5.386066383554456e-06, -0.00011375290004178384, 0.9987389734385891)


In [37]:
p.disconnect()

In [9]:
def create_base_link(xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))    
    return urdfpy.Link("base_link",collisions=[col],inertial=None,visuals=[vis])

def create_arm_link(name,xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None),origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy))    
    return urdfpy.Link(name,collisions=[col],inertial=urdfpy.Inertial(1,[[0, 0, 0],[0, 0, 0],[0, 0, 0]],urdfpy.xyz_rpy_to_matrix(xyz_rpy)),visuals=[vis])

def create_joint(parent,child,xyz_rpy,name,joint_type):
    child.origin = xyz_rpy
    return urdfpy.Joint(name,joint_type=joint_type,parent=parent.name,child=child.name,limit=urdfpy.JointLimit(1,1,-1,1),origin=xyz_rpy)

def create_appendige(base,xyz_rpy,name):
    arm1 = create_arm_link(name+"_arm1",[0, .34, 0,0,0,0])
    joint_base = create_joint(base,arm1,xyz_rpy,name+"_jointBase","fixed")
    arm2 = create_arm_link(name+"_arm2",[0, .34, 0,0,0,0])
    joint1 = create_joint(arm1,arm2,[0, .68, 0,0,0,0],name+"_joint1","revolute")
    return [arm1,arm2],[joint_base,joint1]

base = create_base_link([0,0,0,0,0,0])

print()

links,joints = create_appendige(base,[0,.34,0,0,1,0],"TEST")

links.append(base)

robot = URDF("Model_test",links=links, joints=joints)
robot.save("Models/Model_test.urdf")


In [ ]:
arm1 = create_arm_link("test",[0,.34,0,0,0,0])
base = create_base_link([0,0,0,0,0,0])


# robot = URDF("test",links=[base,arm1], joints=[joint1])

# robot.save("robotTest.urdf")

In [61]:
def generateModels(amountOfModelsToBeGenerated, amountOfMotorOnEachModel):
    for i in range(0,amountOfModelsToBeGenerated):
        jointList = []
        linksList = []
        base = create_base_link([0,0,0,0,0,0])
        linksList.append(base)
        for j in range(amountOfMotorOnEachModel):
            x =r.uniform(-.7,.7) 
            y = r.uniform(-.7,.7)
            z = r.uniform(-.1,.1)
            pitch = r.uniform(-.5,.5) 
            yaw = r.uniform(-.5,.5) 
            roll = r.uniform(-.5,.5) 
            arm1 = create_arm_link("arm"+str(j),[x,y,z,pitch,yaw,roll])
            linksList.append(arm1)
            jointList.append(create_joint(base,arm1,[x,y,z,pitch,yaw,roll],str(j)))
        robot = URDF("Model_"+str(i),links=linksList, joints=jointList)
        robot.save("Models/Model_"+str(i)+".urdf")
generateModels(1,4)

In [64]:
def simulateModels(amountToSimulate, goalOrientation):
    best = ["model-1",100]
    for i in range(0,amountToSimulate):
        physicsClient = p.connect(p.DIRECT)# p.DIRECT for non-graphical version
        p.setAdditionalSearchPath(pybullet_data.getDataPath()) # optionally
        p.setGravity(0,0,-9.8)   
        planeId = p.loadURDF('plane.urdf')
        cubeStartPos = [0,0,1]
        cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
        boxId = p.loadURDF('Models/Model_'+str(i)+'.urdf',cubeStartPos, cubeStartOrientation)
        p.setJointMotorControl2(boxId,0,p.POSITION_CONTROL,1)
        p.setJointMotorControl2(boxId,1,p.POSITION_CONTROL,-1)
        p.setJointMotorControl2(boxId,2,p.POSITION_CONTROL,1)
        cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
        # print("start")
        # print(cubePos,cubeOrn)
        # p.resetDebugVisualizerCamera( cameraDistance=3.5, cameraYaw=30, cameraPitch=52, cameraTargetPosition=cubePos)
        for k in range (300):
            p.stepSimulation()
        #     time.sleep(1./200.)
        cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
        # print("Model_"+str(i))
        # print(np.mean([abs(a - b) for a, b in zip(list(cubeOrn), goalOrientation)]))
        mean = np.mean([abs(a - b) for a, b in zip(list(cubeOrn), goalOrientation)])
        if (mean<best[1]):
            best[1] = mean
            best[0] = "Model_"+str(i)
        p.disconnect()
        # print(cubePos,list(cubeOrn))
    print(best)
    return best

simulateModels(100, [0.050204083161644035, -5.386066383554456e-06, -0.00011375290004178384, 0.9987389734385891])

['Model_86', 0.006028822406110241]


['Model_86', 0.006028822406110241]

https://urdfpy.readthedocs.io/en/latest/generated/urdfpy.Link.html#urdfpy.Link

https://urdfpy.readthedocs.io/en/latest/examples/index.html#loading-from-a-file
